# 数据并行处理

PyTorch非常容易的就可以使用GPU，可以用如下方式把一个模型放到GPU上

```
device = torch.device("cuda：0")
model.to(device)
```

然后可以复制所有的张量到GPU上：

```
mytensor = my_tensor.to(device)
```

请注意，调用 `my_tensor.to(device)` 返回一个GPU上的 `my_tensor` 副本，而不是重写 `my_tensor`。我们需要把它赋值给一个新的张量并在GPU上使用这个张量。

在多GPU上执行前向和反向传播是自然而然的事。然而，PyTorch默认将只是用一个GPU。可以使用 `DataParallel` 让模型并行运行来轻易的让操作在多个GPU上运行。

```
model = nn.DataParallel(model)
```

## 导入和参数

导入PyTorch模块和定义参数

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [2]:
# Parameters 和 DataLoaders
input_size = 5
output_size = 2

batch_size = 30
data_size = 100

设备（Device）：

In [3]:
device = torch.device('cuda: 0' if torch.cuda.is_available() else "cpu")

## 虚拟数据集

要制作一个虚拟（随机）数据集，只需要实现 `__getitem__`。

In [4]:
class RandomDataset(Dataset):
    
    def __init__(self, size, length):
        self.len = length
        self.data = torch.randn(length, size)
        
        
    def __getitem__(self, index):
        return self.data[index]
    
    def __len__(self):
        return self.len
    
rand_loader = DataLoader(dataset=RandomDataset(input_size, data_size), 
                        batch_size=batch_size, shuffle=True)

## 简单模型

作为演示，这里的模型只接受一个输入，执行一个线性操作，然后得到结果。然而，能在任何模型（CNN，RNN，Capsule Net等）上使用 `DataParallel`。

这里，在模型内部放置了一条打印语句来检测输入和输出向量的大小。注意批等级为0时打印的内容。

In [5]:
class Model(nn.Module):
    # Our model
    
    def __init__(self, input_size, output_size):
        super(Model, self).__init__()
        self.fc = nn.Linear(input_size, output_size)
        
    def forward(self, in_put):
        output = self.fc(in_put)
        print('\tIn Model: input size {}, Output size {}'.format(in_put.size(), output.size()))
        
        return output

## 创建一个模型和数据并行

首先，需要创建一个模型实例和检测是否有多个GPU。如果有多个GPU，使用 `nn.DataParallel` 来包装模型。然后通过 `model.to(device)` 把模型放到GPU上。

In [6]:
model = Model(input_size, outpu_size)

if torch.cuda.device_count() > 1:
    print('Let`s use {} GPUs!'.format(torch.cuda.device_count()))
    
    # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
    model = nn.DataParallel(model)
    
model.to(device)

NameError: name 'outpu_size' is not defined

## 运行模型

现在可以看到输入和输出张量的大小

In [ ]:
for data in rand_loader:
    in_put = data.to(device)
    output = model(in_put)
    print('Outside: input_size {}, output_size {}'.format(in_put.size(), output.size()))

## 结果

当我们对30个输入和输出进行批处理时，我们和期望的一样得到30个输入和30个输出，但是若有多个GPU，会得到如下的结果。

### 2个GPU

```
Let's use 2 GPUs!
    In Model： input size torch.Size([15, 5]) output size torch.Size([15, 2])
    In Model： input size torch.Size([15, 5]) output size torch.Size([15, 2])
Outside： input size torch.Size([30, 5]) output_size torch.Size([30, 2])
    In Model： input size torch.Size([15, 5]) output size torch.Size([15, 2])
    In Model： input size torch.Size([15, 5]) output size torch.Size([15, 2])
Outside： input size torch.Size([30, 5]) output_size torch.Size([30, 2])
    In Model： input size torch.Size([15, 5]) output size torch.Size([15, 2])
    In Model： input size torch.Size([15, 5]) output size torch.Size([15, 2])
Outside： input size torch.Size([30, 5]) output_size torch.Size([30, 2])
    In Model： input size torch.Size([5, 5]) output size torch.Size([5, 2])
    In Model： input size torch.Size([5, 5]) output size torch.Size([5, 2])
Outside： input size torch.Size([10, 5]) output_size torch.Size([10, 2])
```

### 3个GPU

```
Let's use 3 GPUs!
    In Model： input size torch.Size([10, 5]) output size torch.Size([10, 2])
    In Model： input size torch.Size([10, 5]) output size torch.Size([10, 2])
    In Model： input size torch.Size([10, 5]) output size torch.Size([10, 2])
Outside： input size torch.Size([30, 5]) output_size torch.Size([30, 2])
    In Model： input size torch.Size([10, 5]) output size torch.Size([10, 2])
    In Model： input size torch.Size([10, 5]) output size torch.Size([10, 2])
    In Model： input size torch.Size([10, 5]) output size torch.Size([10, 2])
Outside： input size torch.Size([30, 5]) output_size torch.Size([30, 2])
    In Model： input size torch.Size([10, 5]) output size torch.Size([10, 2])
    In Model： input size torch.Size([10, 5]) output size torch.Size([10, 2])
    In Model： input size torch.Size([10, 5]) output size torch.Size([10, 2])
Outside： input size torch.Size([30, 5]) output_size torch.Size([30, 2])
    In Model： input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model： input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model： input size torch.Size([2, 5]) output size torch.Size([2, 2])
Outside： input size torch.Size([10, 5]) output_size torch.Size([10, 2])
```

### 8个GPU

```
Let's use 8 GPUs!
    In Model： input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model： input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model： input size torch.Size([2, 5]) output size torch.Size([2, 2])
    In Model： input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model： input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model： input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model： input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model： input size torch.Size([4, 5]) output size torch.Size([4, 2])
Outside： input size torch.Size([30, 5]) output_size torch.Size([30, 2])
    In Model： input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model： input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model： input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model： input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model： input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model： input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model： input size torch.Size([2, 5]) output size torch.Size([2, 2])
    In Model： input size torch.Size([4, 5]) output size torch.Size([4, 2])
Outside： input size torch.Size([30, 5]) output_size torch.Size([30, 2])
    In Model： input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model： input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model： input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model： input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model： input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model： input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model： input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model： input size torch.Size([2, 5]) output size torch.Size([2, 2])
Outside： input size torch.Size([30, 5]) output_size torch.Size([30, 2])
    In Model： input size torch.Size([2, 5]) output size torch.Size([2, 2])
    In Model： input size torch.Size([2, 5]) output size torch.Size([2, 2])
    In Model： input size torch.Size([2, 5]) output size torch.Size([2, 2])
    In Model： input size torch.Size([2, 5]) output size torch.Size([2, 2])
    In Model： input size torch.Size([2, 5]) output size torch.Size([2, 2])
Outside： input size torch.Size([10, 5]) output_size torch.Size([10, 2])
```

## 总结

`DataParallel` 自动的划分数据，并将任务发送到多个GPU上的多个模型。在每个模型完成任务后，`DataParallel` 收集并合并结果返回。